In [7]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [8]:
# Loading the IMDB dataset
df=pd.read_csv('/content/drive/MyDrive/imdb_dataset/imdb_dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [9]:
# Getting info about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [10]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [11]:
# viewing the first 10 elements of the dataset
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Text Processing

In [12]:
# importing libraries
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
stop_words = set(stopwords.words('english'))

cleaning the text

In [16]:
def preprocess_text(text):
  # remove punctuation
  text=text.translate(str.maketrans('','',string.punctuation))
  # convert to lower-case
  text=text.lower()
  # tokenize
  words = word_tokenize(text)
  # remove stopwords
  words=[word for word in words if word not in stop_words]
  return ' '.join(words)

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
df['review']=df['review'].apply(preprocess_text)

In [20]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production br br filming tech...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically theres family little boy jake thinks...,negative
4,petter matteis love time money visually stunni...,positive


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
vectorizer=TfidfVectorizer()
X = vectorizer.fit_transform(df['review'])
y=df['sentiment']

# Split data and train

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
classifier = MultinomialNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

# Model Evaluation

In [27]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score: ", accuracy)

Accuracy score:  0.8699


In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.86      0.88      0.87      4961
    positive       0.88      0.86      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/imdb_dataset/imdb_dataset.csv')

# Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha()]
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['review'] = df['review'].apply(preprocess_text)

# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

# Model training and evaluation
models = [
    ('SVM', SVC()),
    ('Naive Bayes', MultinomialNB()),
    ('Random Forest', RandomForestClassifier())
]

best_accuracy = 0
best_model = None
best_model_name = None

for name, model in models:
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy}")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = pipeline
        best_model_name = name

print(f"Best model: {best_model_name} with Accuracy: {best_accuracy}")
print("Classification Report:")
print(classification_report(y_test, best_model.predict(X_test)))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


SVM Accuracy: 0.8989
Naive Bayes Accuracy: 0.8657
Random Forest Accuracy: 0.8551
Best model: SVM with Accuracy: 0.8989
Classification Report:
              precision    recall  f1-score   support

    negative       0.91      0.88      0.90      4961
    positive       0.89      0.91      0.90      5039

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



In [31]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

# Load the dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=20000)

# Pad sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# Build the model
model = Sequential()
model.add(Embedding(20000, 128, input_length=max_review_length))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)

# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

17464789/17464789 [==============================] - 0s 0us/step
Epoch 1/5
391/391 [==============================] - 188s 478ms/step - loss: 0.3999 - accuracy: 0.8101 - val_loss: 0.2928 - val_accuracy: 0.8765
Epoch 2/5
391/391 [==============================] - 205s 524ms/step - loss: 0.0667 - accuracy: 0.9785 - val_loss: 0.3677 - val_accuracy: 0.8635
Epoch 3/5
391/391 [==============================] - 217s 557ms/step - loss: 0.0043 - accuracy: 0.9996 - val_loss: 0.4302 - val_accuracy: 0.8709
Epoch 4/5
391/391 [==============================] - 198s 506ms/step - loss: 4.4543e-04 - accuracy: 1.0000 - val_loss: 0.4764 - val_accuracy: 0.8712
Epoch 5/5
391/391 [==============================] - 208s 532ms/step - loss: 1.5233e-04 - accuracy: 1.0000 - val_loss: 0.5089 - val_accuracy: 0.8721
Accuracy: 87.21%
